In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

In [ ]:
from openmdao.utils.assert_utils import assert_near_equal

import os
if os.path.exists('cases.sql'):
    os.remove('cases.sql')

# Case Recording

## Case Recording Options

The available recording options for different OpenMDAO classes are summarized in this table:

|Recording Options        |Driver|System|Solver|Problem|
|:------------------------|:----:|:----:|:----:|:-----:|
|record_constraints       |  X   |      |      |  X    |
|record_desvars           |  X   |      |      |  X    |
|record_objectives        |  X   |      |      |  X    |
|record_derivatives       |  X   |      |      |  X    |
|record_responses         |  X   |      |      |  X    |
|record_inputs            |  X   |  X   |  X   |  X    |
|record_outputs           |  X   |  X   |  X   |  X    |
|record_residuals         |  X   |  X   |      |  X    |
|record_abs_error         |      |      |  X   |       |
|record_rel_error         |      |      |  X   |       |
|record_solver_residuals  |      |      |  X   |       |
|includes                 |  X   |  X   |  X   |  X    |
|excludes                 |  X   |  X   |  X   |  X    |
|options_excludes         |      |  X   |      |       |

## Specifying a Case Prefix
It is possible to record data from multiple executions by specifying a prefix that will be used to differentiate the cases. This prefix can be specified when calling run_model or run_driver and will be prepended to the case ID in the recorded case data:

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src87", get_code("openmdao.test_suite.components.sellar_feature.SellarDerivatives"), display=False)

:::{Admonition} `SellarDerivatives` class definition 
:class: dropdown

{glue:}`code_src87`
:::

In [ ]:
import openmdao.api as om
from openmdao.test_suite.components.sellar_feature import SellarDerivatives

prob = om.Problem(model=SellarDerivatives())
prob.setup()

recorder = om.SqliteRecorder("cases.sql", record_viewer_data=False)
prob.model.add_recorder(recorder)
prob.driver.add_recorder(recorder)

prob.set_solver_print(0)

prob.run_model(case_prefix='Model_Run1')
prob.run_driver(case_prefix='Driver_Run1')

prob.run_model('Model_Run2')
prob.run_driver('Driver_Run2')

prob.cleanup()

cr = om.CaseReader("cases.sql")

# all cases recorded by the root system
model_cases = cr.list_cases('root', recurse=False)

In [ ]:
assert '\n'.join(model_cases) == '\n'.join([
    'Model_Run1_rank0:root._solve_nonlinear|0',
    'Driver_Run1_rank0:Driver|0|root._solve_nonlinear|0',
    'Model_Run2_rank0:root._solve_nonlinear|0',
    'Driver_Run2_rank0:Driver|0|root._solve_nonlinear|0'
])

In [ ]:
# all cases recorded by the driver
driver_cases = cr.list_cases('driver', recurse=False)

In [ ]:
assert '\n'.join(driver_cases) == '\n'.join([
    'Driver_Run1_rank0:Driver|0', 
    'Driver_Run2_rank0:Driver|0'
])

import os
if os.path.exists('cases.sql'):
    os.remove('cases.sql')

```{note}
A recorder can be attached to more than one object. Also, more than one recorder can be attached to an object.
```
```{note}
In this example, we have disabled the saving of data needed by the standalone N2 visualizer and debugging tool by setting `record_viewer_data` to `False`.
```

# Recording Options Include and Exclude Matching

The `includes` and `excludes` recording options provide support for Unix shell-style wildcards, which are not 
the same as regular expressions. The documentation for the `fnmatchcase` function from the Python standard library
documents the wildcards.

# Recording Options Precedence

The precedence of recording options that determines what gets recorded can sometime be a little confusing. 
Here is an example that might help. The code shows how the record_desvars and includes options interact.

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src88", get_code("openmdao.test_suite.components.paraboloid.Paraboloid"), display=False)

:::{Admonition} `Paraboloid` class definition 
:class: dropdown

{glue:}`code_src88`
:::

In [ ]:
import openmdao.api as om
from openmdao.test_suite.components.paraboloid import Paraboloid

prob = om.Problem()
prob.driver = om.ScipyOptimizeDriver(optimizer='SLSQP', tol=1e-9, disp=False)

model = prob.model

model.add_subsystem('comp', Paraboloid(), promotes=['*'])
model.add_subsystem('con', om.ExecComp('c = x - y'), promotes=['*'])

model.set_input_defaults('x', val=50.0)
model.set_input_defaults('y', val=50.0)

model.add_design_var('x', lower=-50.0, upper=50.0)
model.add_design_var('y', lower=-50.0, upper=50.0)
model.add_objective('f_xy')
model.add_constraint('c', lower=15.0)

case_file = 'cases.sql'

In [ ]:
# First case with record_desvars = True and includes = []
recorder = om.SqliteRecorder(case_file)
prob.driver.add_recorder(recorder)
prob.driver.recording_options['record_desvars'] = True
prob.driver.recording_options['record_outputs'] = True
prob.driver.recording_options['includes'] = []
prob.driver.recording_options['excludes'] = []

prob.setup()
prob.run_driver()
prob.cleanup()

cr = om.CaseReader(case_file)
case = cr.get_case(0)

case.outputs

In [ ]:
assert sorted(case.outputs.keys()) == ['c', 'f_xy', 'x', 'y']

import os
if os.path.exists('cases.sql'):
    os.remove('cases.sql')

In [ ]:
# Second case with record_desvars = False and includes = []
recorder = om.SqliteRecorder(case_file)
prob.driver.add_recorder(recorder)
prob.driver.recording_options['record_desvars'] = False
prob.driver.recording_options['record_outputs'] = True
prob.driver.recording_options['includes'] = []

prob.setup()
prob.run_driver()
prob.cleanup()

cr = om.CaseReader(case_file)
case = cr.get_case(0)

case.outputs

In [ ]:
assert sorted(case.outputs.keys()) == ['c', 'f_xy']

import os
if os.path.exists('cases.sql'):
    os.remove('cases.sql')

In [ ]:
# Third case with record_desvars = True and includes = ['*']
recorder = om.SqliteRecorder(case_file)
prob.driver.add_recorder(recorder)
prob.driver.recording_options['record_desvars'] = True
prob.driver.recording_options['record_outputs'] = True
prob.driver.recording_options['includes'] = ['*']

prob.setup()
prob.run_driver()
prob.cleanup()

cr = om.CaseReader(case_file)
case = cr.get_case(0)

case.outputs

In [ ]:
assert sorted(case.outputs.keys()) == ['c', 'f_xy', 'x', 'y']

import os
if os.path.exists('cases.sql'):
    os.remove('cases.sql')

In [ ]:
# Fourth case with record_desvars = False, record_outputs = True, and includes = ['*']
recorder = om.SqliteRecorder(case_file)
prob.driver.add_recorder(recorder)
prob.driver.recording_options['record_desvars'] = False
prob.driver.recording_options['record_outputs'] = True
prob.driver.recording_options['includes'] = ['*']

prob.setup()
prob.run_driver()
prob.cleanup()

cr = om.CaseReader(case_file)
case = cr.get_case(0)

case.outputs

In [ ]:
assert sorted(case.outputs.keys()) == ['c', 'f_xy', 'x', 'y']